## Config

In [1]:
import os
import argparse
from datetime import datetime
from collections import defaultdict
from datetime import datetime
from pathlib import Path
import pprint
from torch import optim
import torch.nn as nn

In [2]:
# path to a pretrained word embedding file
# word_emb_path = '/mnt/soyeon/workspace/glove.840B.300d.txt'
word_emb_path = '/home/ubuntu/soyeon/glove.840B.300d.txt'
assert(word_emb_path is not None)


# username = Path.home().name
# project_dir = Path(__file__).resolve().parent.parent
# sdk_dir = project_dir.joinpath('CMU-MultimodalSDK')
# data_dir = project_dir.joinpath('datasets')

sdk_dir = Path('/home/ubuntu/soyeon/CMU-MultimodalSDK')
data_dir = Path('/home/ubuntu/soyeon/MSIR/datasets')
data_dict = {'mosi': data_dir.joinpath('MOSI'), 'mosei': data_dir.joinpath(
    'MOSEI'), 'ur_funny': data_dir.joinpath('UR_FUNNY')}
optimizer_dict = {'RMSprop': optim.RMSprop, 'Adam': optim.Adam}
activation_dict = {'elu': nn.ELU, "hardshrink": nn.Hardshrink, "hardtanh": nn.Hardtanh,
                   "leakyrelu": nn.LeakyReLU, "prelu": nn.PReLU, "relu": nn.ReLU, "rrelu": nn.RReLU,
                   "tanh": nn.Tanh}

In [3]:
def str2bool(v):
    """string to boolean"""
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


class Config(object):
    def __init__(self, **kwargs):
        """Configuration Class: set kwargs as class attributes with setattr"""
        if kwargs is not None:
            for key, value in kwargs.items():
                if key == 'optimizer':
                    value = optimizer_dict[value]
                if key == 'activation':
                    value = activation_dict[value]
                setattr(self, key, value)

        # Dataset directory: ex) ./datasets/cornell/
        self.dataset_dir = data_dict[self.data.lower()]
        self.sdk_dir = sdk_dir
        # Glove path
        self.word_emb_path = word_emb_path

        # Data Split ex) 'train', 'valid', 'test'
        # self.data_dir = self.dataset_dir.joinpath(self.mode)
        self.data_dir = self.dataset_dir

    def __str__(self):
        """Pretty-print configurations in alphabetical order"""
        config_str = 'Configurations\n'
        config_str += pprint.pformat(self.__dict__)
        return config_str

In [4]:
import easydict

def get_config(parse=True, **optional_kwargs):
    time_now = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    kwargs = easydict.EasyDict({
        # Mode
        'mode': 'train',
        'runs': 5,

        # Bert
        'use_bert': True,
        'use_cmd_sim': True,

        # Train
        'name': f"{time_now}",
        'num_classes': 0,
        'batch_size': 128,
        'eval_batch_size': 10,
        'n_epoch': 500,
        'patience': 6,
        
        'diff_weight': 0.3,
        'sim_weight': 1.0,
        'sp_weight': 0.0,
        'recon_weight': 1.0,

        'learning_rate': 1e-4,
        'optimizer': 'Adam',
        'clip': 1.0,

        'rnncell': 'lstm',
        'embedding_size': 300,
        'hidden_size': 128,
        'dropout': 0.5,
        'reverse_grad_weight': 1.0,
        
        # Selection activation from 'elu', 'hardshrink', 'hardtanh', 'leakyrelu', 'prelu', 'relu', 'rrelu', 'tanh'
        'activation': 'relu',

        # Model
        'model': 'MISA',
        
        # Data
        'data': 'mosi'
    })

    if kwargs.data == "mosi":
        kwargs.num_classes = 1
        kwargs.batch_size = 64
    elif kwargs.data == "mosei":
        kwargs.num_classes = 1
        kwargs.batch_size = 16
    elif kwargs.data == "ur_funny":
        kwargs.num_classes = 2
        kwargs.batch_size = 32
    else:
        print("No dataset mentioned")
        exit()

    return Config(**kwargs)

## Train

In [5]:
import os
import pickle
from pyexpat import model
import numpy as np
from random import random

# from config import get_config, activation_dict
from data_loader import get_loader
from solver import Solver
from test_instance import TestMOSI, TestMOSEI

import torch
import torch.nn as nn
from torch.nn import functional as F

/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/ubuntu/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /ho

In [6]:
# Setting random seed
random_name = str(random())
random_seed = 336   
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

# Setting the config for each stage
train_config = get_config(mode='train')
dev_config = get_config(mode='dev')
test_config = get_config(mode='test')


print(train_config)


Configurations
{'activation': <class 'torch.nn.modules.activation.ReLU'>,
 'batch_size': 64,
 'clip': 1.0,
 'data': 'mosi',
 'data_dir': PosixPath('/home/ubuntu/soyeon/MSIR/datasets/MOSI'),
 'dataset_dir': PosixPath('/home/ubuntu/soyeon/MSIR/datasets/MOSI'),
 'diff_weight': 0.3,
 'dropout': 0.5,
 'embedding_size': 300,
 'eval_batch_size': 10,
 'hidden_size': 128,
 'learning_rate': 0.0001,
 'mode': 'train',
 'model': 'MISA',
 'n_epoch': 500,
 'name': '2022-06-06_23:13:46',
 'num_classes': 1,
 'optimizer': <class 'torch.optim.adam.Adam'>,
 'patience': 6,
 'recon_weight': 1.0,
 'reverse_grad_weight': 1.0,
 'rnncell': 'lstm',
 'runs': 5,
 'sdk_dir': PosixPath('/home/ubuntu/soyeon/CMU-MultimodalSDK'),
 'sim_weight': 1.0,
 'sp_weight': 0.0,
 'use_bert': True,
 'use_cmd_sim': True,
 'word_emb_path': '/home/ubuntu/soyeon/glove.840B.300d.txt'}


In [7]:
# Creating pytorch dataloaders
train_data_loader = get_loader(train_config, shuffle = True)
dev_data_loader = get_loader(dev_config, shuffle = False)
test_data_loader = get_loader(test_config, shuffle = False)

train
train
train


In [8]:
# Solver is a wrapper for model traiing and testing
solver = Solver
solver = solver(train_config, dev_config, test_config, train_data_loader, dev_data_loader, test_data_loader, is_train=True)

In [9]:
model = solver.build()

Build Graph


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/res

	bertmodel.embeddings.word_embeddings.weight True
	bertmodel.embeddings.position_embeddings.weight True
	bertmodel.embeddings.token_type_embeddings.weight True
	bertmodel.embeddings.LayerNorm.weight True
	bertmodel.embeddings.LayerNorm.bias True
	bertmodel.encoder.layer.0.attention.self.query.weight True
	bertmodel.encoder.layer.0.attention.self.query.bias True
	bertmodel.encoder.layer.0.attention.self.key.weight True
	bertmodel.encoder.layer.0.attention.self.key.bias True
	bertmodel.encoder.layer.0.attention.self.value.weight True
	bertmodel.encoder.layer.0.attention.self.value.bias True
	bertmodel.encoder.layer.0.attention.output.dense.weight True
	bertmodel.encoder.layer.0.attention.output.dense.bias True
	bertmodel.encoder.layer.0.attention.output.LayerNorm.weight True
	bertmodel.encoder.layer.0.attention.output.LayerNorm.bias True
	bertmodel.encoder.layer.0.intermediate.dense.weight True
	bertmodel.encoder.layer.0.intermediate.dense.bias True
	bertmodel.encoder.layer.0.output.dens

In [10]:
solver.train()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training Start!


/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/si

Training loss: 4.3373
Current patience: 6, current trial: 1.
Found new best model on dev set!
Training loss: 3.7146
Current patience: 6, current trial: 1.
Training loss: 3.4759
Current patience: 5, current trial: 1.
Training loss: 3.4116
Current patience: 4, current trial: 1.
Training loss: 3.1194
Current patience: 3, current trial: 1.
Training loss: 3.0754
Current patience: 2, current trial: 1.
Training loss: 3.0093
Current patience: 1, current trial: 1.
Training loss: 2.8166
Current patience: 0, current trial: 1.
Running out of patience, loading previous best model.
Current learning rate: 5e-05
Running out of patience, early stopping.
mae:  1.3133608
corr:  0.11986355497841569
mult_acc:  0.19251753702260327
Classification Report (pos/neg) :
              precision    recall  f1-score   support

       False    0.00000   0.00000   0.00000       552
        True    0.55122   1.00000   0.71069       678

    accuracy                        0.55122      1230
   macro avg    0.27561   0.5

/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
torch.save(model.state_dict(), "./saved_models_MISA_mosi.pt")

In [10]:
model.load_state_dict(torch.load("./saved_models_MISA_mosi.pt"))
model.eval()

MISA(
  (activation): ReLU()
  (tanh): Tanh()
  (bertmodel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [11]:
# Make test result file by instance
tester = TestMOSI
tester = tester(model)
segment_list, preds, preds_2, preds_7 = tester.start()

  0%|          | 0/69 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/anaconda3/envs/pytorch1.7.1_p37/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 69/69 [00:0

0 th data
c7UH_rxdZv4[0]
0.3923420011997223
positive
Neutral
1 th data
c7UH_rxdZv4[1]
0.2903745770454407
positive
Neutral
2 th data
c7UH_rxdZv4[2]
0.19309017062187195
positive
Neutral
3 th data
c7UH_rxdZv4[3]
0.26682808995246887
positive
Neutral
4 th data
c7UH_rxdZv4[4]
0.2718200981616974
positive
Neutral
5 th data
c7UH_rxdZv4[5]
0.2544107735157013
positive
Neutral
6 th data
c7UH_rxdZv4[6]
0.16008231043815613
positive
Neutral
7 th data
c7UH_rxdZv4[7]
0.29097780585289
positive
Neutral
8 th data
c7UH_rxdZv4[8]
0.21132530272006989
positive
Neutral
9 th data
c7UH_rxdZv4[9]
0.2739690840244293
positive
Neutral
10 th data
c7UH_rxdZv4[10]
0.24210606515407562
positive
Neutral
11 th data
c7UH_rxdZv4[11]
0.2579873204231262
positive
Neutral
12 th data
c7UH_rxdZv4[12]
0.25383099913597107
positive
Neutral
13 th data
c7UH_rxdZv4[13]
0.24042315781116486
positive
Neutral
14 th data
c7UH_rxdZv4[14]
0.2915196120738983
positive
Neutral
15 th data
c7UH_rxdZv4[15]
0.24061758816242218
positive
Neutral
16 th 

In [12]:
len(segment_list)

685

In [14]:
import pickle
# Gold-truth
labels = []
labels_2 = []
labels_7 = []
with open(f"../../datasets/MOSI/mosi.pkl", "rb") as handle:
    data = pickle.load(handle)

test_data = data["test"]

video = set()
count = 0

for idx in range(len(test_data)):
    (words, visual, acoustic), label, segment = test_data[idx]
    # if args.dataset == 'mosi':
    assert segment_list[idx] == segment
    # else:
    #     video_name = segment[0]
    #     if video_name in video:
    #         count += 1
    #     else:
    #         video.add(video_name)
    #         count = 0
    #     assert segment_list[idx] == segment

    labels.append(label[0][0])

    # label_2 appending
    if label > 0:
        labels_2.append('positive')
    else:
        labels_2.append('negative')
    
    # label_7 appending
    if label < -15/7:
        labels_7.append('very negative')
    elif label < -9/7:
        labels_7.append('negative')
    elif label < -3/7:
        labels_7.append('slightly negative')
    elif label < 3/7:
        labels_7.append('Neutral')
    elif label < 9/7:
        labels_7.append('slightly positive')
    elif label < 15/7:
        labels_7.append('positive')
    else:
        labels_7.append('very positive')
count = 0

In [1]:
import plotly.express as px
import plotly.subplots as sp
import pandas as pd

d = {'segmentID': segment_list, 'labels': labels, 'labels_2': labels_2, 'labels_7': labels_7, 'preds': preds, 'preds_2': preds_2, 'preds_7': preds_7}
df = pd.DataFrame(data=d)
order = ['very negative', 'negative', 'slightly negative', 'Neutral', 'slightly positive', 'positive', 'very positive']

fig1 = px.bar(df, x="labels_7")
fig2 = px.bar(df, x="preds_7")

fig1_traces = []
fig2_traces = []

for trace in range(len(fig1["data"])):
    fig1_traces.append(fig1["data"][trace])
for trace in range(len(fig2["data"])):
    fig2_traces.append(fig2["data"][trace])

this_figure = sp.make_subplots(rows=1, cols=2, subplot_titles=("Gold", "MISA"))
for traces in fig1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in fig2_traces:
    this_figure.append_trace(traces, row=1, col=2)

# this_figure.update_layout(height=600, width=1500, title_text="CMU-MOSI 7 Class Sentiment Intensity")
this_figure.update_xaxes(categoryorder='array', categoryarray= order)
# this_figure.update_yaxes(range=[0,250])
this_figure.show()

NameError: name 'segment_list' is not defined